# Computing Sharpe Ratio

In [ ]:
from AlgorithmImports import *
from datetime import datetime, date, time, timedelta
import numpy as np
import pandas as pd
import statsmodels
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint, adfuller
import matplotlib.pyplot as plt
from scipy import stats

qb = QuantBook()
start_date = datetime(2019,2,1,0,0)
end_date = datetime(2021,2,1,0,0)

Compute the Sharpe Ratio of a buy-and-hold strategy for each of the above stocks individually for the time period 2/1/2019 to 2/1/2021.

In [ ]:
tickers = ['GS', 'MS', 'AMD', 'XOM']
for ticker in tickers:
    qb.AddEquity(ticker, Resolution.Daily)
    history = qb.History(tickers, start=start_date, end=end_date, resolution=Resolution.Daily)
    df = history['close'].unstack(level=0)
    dg = df[ticker].apply(lambda x: math.log(x))
    dg = dg.diff(1).dropna() # daily log return

    #### ***Compute Daily Return, Convert to Annual Return, compute Sharpe Ratio***
    # Annual Return
    x_0 = df[ticker].iloc[0] # first price
    x_1 = df[ticker].iloc[-1] # latest prices

    return_yr = (x_1-x_0)/x_0

    sigma_d = dg.std()
    sigma_yr = sigma_d*math.sqrt(260) # square root of time rule

    rf = 0.0025

    sharpe = (return_yr - rf)/sigma_yr
    print("Sharpe Ratio for %s:" % ticker, sharpe)

Perform Jarque-Bera test to see if Geometric Brownian Motion assumption holds for these stocks during the given time period. 95% confidence interval assumed.

In [ ]:
df = history['close'].unstack(level=0)
df.columns = tickers

for ticker in tickers:
    dg = df[ticker].apply(lambda x: math.log(x))
    dg = dg.diff(1).dropna() # daily log return
    jarquebera = statsmodels.stats.stattools.jarque_bera(dg.values)
    print(ticker, ":", jarquebera)